In [9]:
#1.获取promoter和tf-seq,等信息

In [46]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/zlab/boltzscan')
from Bio import SeqIO
from fimocistarget.fimo_cistarg import PyMEMESuiteCisTargetBuilder
from fimocistarget.cistarg_impl import GenomeBackgroundComparator, remove_redundant_intervals_per_motif
import pandas as pd
from Bio.Seq import Seq
from pathlib import Path
from utils.io_utils import get_boltz_res, pwm_to_meme, read_motif
from cropd2p.cropdock.a3m import read_a3m
from utils.utils import calc_ipsae
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
extend_range = 5

pwm_dir = Path('/home/zlab/ms/odata/OB_rose_pwms')
tf2pwm_file = pwm_dir / 'tf2pwms.json'

with open(tf2pwm_file, 'r') as f:
    tf2pwm = json.load(f)

In [48]:
pwm2tf = {}
for tf, pwms in tf2pwm.items():
    for pwm in pwms:
        if pwm not in pwm2tf:
            pwm2tf[pwm] = []
        pwm2tf[pwm].append(tf)

In [5]:
tf_msa_dir = Path('data/MSA_DATABASE/rose_msa')
promoter_file = 'input_other/rose/Rosa_chinensis_promoter.fasta'
tf_msa_dct = {f.parts[-2]: f for f in tf_msa_dir.rglob('*0.a3m')}
# pep_seqs = SeqIO.to_dict(SeqIO.parse('/home/zlab/boltzscan/test_cgy/data/Target_Protein_CiJMJ19.fa', 'fasta'))
# tf_seqs = SeqIO.to_dict(SeqIO.parse('input_other/dma/Rb.pep.fa', 'fasta'))
promoter_seqs = SeqIO.to_dict(SeqIO.parse(promoter_file, 'fasta'))

In [26]:
t_name = pd.read_csv('purple_tf_{source}.csv')['t_name'].tolist()
t_name = [x.replace('mRNA:RchiOBHm','RchiOBHm_') for x in t_name]
t_name = [x.replace('ncRNA:RchiOBHm','RchiOBHm_') for x in t_name]


need_promoter_seq = {t:promoter_seqs[t] for t in t_name if t in promoter_seqs}

In [34]:
with open('input_other/cxq/t_name_promoters.fasta', 'w') as f:
    for k,v in need_promoter_seq.items():
        f.write(f'>{k}')
        f.write('\n')
        f.write(str(v.seq))
        f.write('\n')

promoter_file = 'input_other/cxq/t_name_promoters.fasta'

### 在没有启动子序列的情况下，可以自己去提取

In [14]:
# gene_id = 'RchiOBHm_Chr3g0471271'
# promoter_seqs[gene_id]

# with open(f'{gene_id}_promoter.fasta','w') as f:
#     SeqIO.write(promoter_seqs[gene_id], f, 'fasta')


In [50]:
A = 3.082e-01
C = 1.918e-01
G = 1.918e-01
T = 3.082e-01
save_meme_dir = Path('data/PWM_DATABASE/rose_OB_meme')
for t,ps in tf2pwm.items():
    for p in ps:
        try:
            pwm_f = pwm_dir / f'{p}.txt'
            m_info = read_motif(pwm_f)
            mid = m_info['name']
            pwm_matrix = m_info['motif']
            save_dir = save_meme_dir
            save_dir.mkdir(exist_ok=True, parents=True)
            pwm_to_meme(pwm=pwm_matrix, motif_id=p, nsites=100, output_file=save_dir / f'{p}.meme',
            background=[A,C,G,T])
        except:
             pass

INFO:utils.io_utils:成功将基序 'M07626_3.00' 写入文件: data/PWM_DATABASE/rose_OB_meme/M07626_3.00.meme
INFO:utils.io_utils:成功将基序 'M01372_3.00' 写入文件: data/PWM_DATABASE/rose_OB_meme/M01372_3.00.meme
INFO:utils.io_utils:成功将基序 'M07627_3.00' 写入文件: data/PWM_DATABASE/rose_OB_meme/M07627_3.00.meme
INFO:utils.io_utils:成功将基序 'M01375_3.00' 写入文件: data/PWM_DATABASE/rose_OB_meme/M01375_3.00.meme
INFO:utils.io_utils:成功将基序 'rose_RcHm_v2.0_Chr0c19g0500091_0.07_pwm' 写入文件: data/PWM_DATABASE/rose_OB_meme/rose_RcHm_v2.0_Chr0c19g0500091_0.07_pwm.meme
INFO:utils.io_utils:成功将基序 'M07201_3.00' 写入文件: data/PWM_DATABASE/rose_OB_meme/M07201_3.00.meme
INFO:utils.io_utils:成功将基序 'M07227_3.00' 写入文件: data/PWM_DATABASE/rose_OB_meme/M07227_3.00.meme
INFO:utils.io_utils:成功将基序 'M07228_3.00' 写入文件: data/PWM_DATABASE/rose_OB_meme/M07228_3.00.meme
INFO:utils.io_utils:成功将基序 'M07200_3.00' 写入文件: data/PWM_DATABASE/rose_OB_meme/M07200_3.00.meme
INFO:utils.io_utils:成功将基序 'M07230_3.00' 写入文件: data/PWM_DATABASE/rose_OB_meme/M07230_3.00.meme
INFO

In [51]:
fimo = PyMEMESuiteCisTargetBuilder(
    fasta_file=promoter_file,
    motif_dir = save_meme_dir,
    output_dir='input_other/cxq/cisdb',
    pvalue_thresh=1e-5,
    max_stored_scores=500000,
    motif_pseudo=0.1,
    custom_bg= {'A':A, 'C':C, 'G':G, 'T':T}
)
fimo.build_database()

INFO:fimocistarget.fimo_cistarg:初始化完成
INFO:fimocistarget.fimo_cistarg:FASTA文件: input_other/cxq/t_name_promoters.fasta
INFO:fimocistarget.fimo_cistarg:Motif目录: data/PWM_DATABASE/rose_OB_meme
INFO:fimocistarget.fimo_cistarg:P-value阈值: 1e-05
INFO:fimocistarget.fimo_cistarg:
INFO:fimocistarget.fimo_cistarg:开始构建cisTarget数据库
INFO:fimocistarget.fimo_cistarg:======================================================================
INFO:fimocistarget.fimo_cistarg:找到 1934 个motif文件
INFO:fimocistarget.fimo_cistarg:加载了 231 个序列
INFO:fimocistarget.fimo_cistarg:======================================================================
INFO:fimocistarget.fimo_cistarg:开始运行FIMO扫描...
INFO:fimocistarget.fimo_cistarg:======================================================================
INFO:fimocistarget.fimo_cistarg:
[1/1934] 处理文件: M00003_3.00.meme
INFO:fimocistarget.fimo_cistarg:处理motif文件: M00003_3.00.meme
INFO:fimocistarget.fimo_cistarg:  - 找到 124 个匹配
INFO:fimocistarget.fimo_cistarg:
[2/1934] 处理文件: M00004_3.00

(motif_id               M00003_3.00  M00004_3.00  M00005_3.00  M00006_3.00  \
 sequence_name                                                               
 RchiOBHm_Chr1g0314821    10.472973     0.000000    10.782609     0.000000   
 RchiOBHm_Chr1g0314871    10.351351     0.000000     0.000000     0.000000   
 RchiOBHm_Chr1g0326551    11.310811     0.000000    12.855072     0.000000   
 RchiOBHm_Chr1g0329541     0.000000     0.000000     0.000000    11.630252   
 RchiOBHm_Chr1g0333261     0.000000    11.132075     0.000000     0.000000   
 ...                            ...          ...          ...          ...   
 RchiOBHm_Chr7g0219651     0.000000     0.000000     0.000000     0.000000   
 RchiOBHm_Chr7g0219681     0.000000    10.820755     0.000000     0.000000   
 RchiOBHm_Chr7g0224061     0.000000    10.584906     0.000000    10.722689   
 RchiOBHm_Chr7g0224111    10.337838    10.311321     0.000000     0.000000   
 RchiOBHm_Chr7g0239251     0.000000     0.000000     0.000000   

In [52]:
hit = pd.read_csv('input_other/cxq/cisdb/fimo_results_raw.csv.gz')
hit

,sequence_name,start,stop,strand,score,pvalue,qvalue,motif_id
0,RchiOBHm_Chr5g0010361,1165,1174,+,17.702703,2.822224e-07,0.197502,M00003_3.00
1,RchiOBHm_Chr3g0449981,573,582,+,17.594595,3.904829e-07,0.197502,M00003_3.00
2,RchiOBHm_Chr2g0124691,1916,1907,-,16.891892,1.017943e-06,0.237455,M00003_3.00
3,RchiOBHm_Chr5g0060001,66,75,+,16.459459,1.300165e-06,0.237455,M00003_3.00
4,RchiOBHm_Chr3g0449981,2173,2164,-,16.459459,1.300165e-06,0.237455,M00003_3.00
...,...,...,...,...,...,...,...,...
234528,RchiOBHm_Chr6g0283981,1497,1504,+,9.128571,8.808195e-05,1.000000,rose_RcHm_v2.0_Chr7g0243841_0.17_pwm
234529,RchiOBHm_Chr6g0304081,1532,1525,-,9.128571,8.808195e-05,1.000000,rose_RcHm_v2.0_Chr7g0243841_0.17_pwm
234530,RchiOBHm_Chr2g0140011,1594,1601,+,9.128571,8.808195e-05,1.000000,rose_RcHm_v2.0_Chr7g0243841_0.17_pwm
234531,RchiOBHm_Chr6g0283511,2118,2125,+,9.128571,8.808195e-05,1.000000,rose_RcHm_v2.0_Chr7g0243841_0.17_pwm


In [59]:
hit_cleand = remove_redundant_intervals_per_motif(hit, .5)
fimo_df = hit_cleand[hit_cleand['pvalue'] <= 1e-4]
fimo_df.shape

(195775, 8)

In [60]:
# 生成TF蛋白和启动子的序列字段
# fimo_df['tf_seq'] = fimo_df['motif_id'].apply(lambda x: str(tf_seqs[x].seq))
fimo_df['promoter_seq'] = fimo_df['sequence_name'].apply(lambda x: str(promoter_seqs[x].seq))


In [61]:
fimo_df['tf_name'] = fimo_df['motif_id'].map(pwm2tf)
fimo_df_split =  fimo_df.explode('tf_name')

fimo_df_split

,sequence_name,start,stop,strand,score,pvalue,qvalue,motif_id,promoter_seq,tf_name
120504,RchiOBHm_Chr1g0314821,28,48,+,14.945946,0.000006,0.659401,M07809_3.00,CAATGAACTAAGTTTGGATCTTTCTTGGTTGATTCTATGTTGACTT...,RcHm_v2.0_Chr2g0158801
120504,RchiOBHm_Chr1g0314821,28,48,+,14.945946,0.000006,0.659401,M07809_3.00,CAATGAACTAAGTTTGGATCTTTCTTGGTTGATTCTATGTTGACTT...,RcHm_v2.0_Chr3g0485691
120504,RchiOBHm_Chr1g0314821,28,48,+,14.945946,0.000006,0.659401,M07809_3.00,CAATGAACTAAGTTTGGATCTTTCTTGGTTGATTCTATGTTGACTT...,RcHm_v2.0_Chr3g0485711
120504,RchiOBHm_Chr1g0314821,28,48,+,14.945946,0.000006,0.659401,M07809_3.00,CAATGAACTAAGTTTGGATCTTTCTTGGTTGATTCTATGTTGACTT...,RcHm_v2.0_Chr5g0002561
120504,RchiOBHm_Chr1g0314821,28,48,+,14.945946,0.000006,0.659401,M07809_3.00,CAATGAACTAAGTTTGGATCTTTCTTGGTTGATTCTATGTTGACTT...,RcHm_v2.0_Chr5g0040801
...,...,...,...,...,...,...,...,...,...,...
23718,RchiOBHm_Chr7g0239251,2134,2125,-,10.731707,0.000048,0.913566,M01728_3.00,GGAAACTAACAACCAATATGCTTCGTCACATTTCAATTAAAGAACC...,RcHm_v2.0_Chr4g0435771
78185,RchiOBHm_Chr7g0239251,2154,2140,-,7.608108,0.000076,0.275726,M07220_3.00,GGAAACTAACAACCAATATGCTTCGTCACATTTCAATTAAAGAACC...,RcHm_v2.0_Chr2g0102031
78185,RchiOBHm_Chr7g0239251,2154,2140,-,7.608108,0.000076,0.275726,M07220_3.00,GGAAACTAACAACCAATATGCTTCGTCACATTTCAATTAAAGAACC...,RcHm_v2.0_Chr3g0455311
78185,RchiOBHm_Chr7g0239251,2154,2140,-,7.608108,0.000076,0.275726,M07220_3.00,GGAAACTAACAACCAATATGCTTCGTCACATTTCAATTAAAGAACC...,RcHm_v2.0_Chr3g0469381


In [64]:
with open('input_other/cxq/down_genes.txt', 'w') as f:
    for i in fimo_df_split[fimo_df_split['tf_name'] == 'RcHm_v2.0_Chr3g0461481']['sequence_name'].unique():
        f.write(i)
        f.write('\n')

In [21]:
# 重置索引（可选，但推荐，否则索引会重复）
fimo_df_split = fimo_df_split.reset_index(drop=True)

# 查看结果
fimo_df_split['boltz_name'] = fimo_df_split['tf_name'] + '-' + fimo_df_split['motif_id'] + '-' + fimo_df_split['sequence_name'] + '-' + fimo_df_split['start'].astype(str) + '-' + fimo_df_split['stop'].astype(str)
fimo_df_split['tf_name'].unique().shape

(166,)

In [22]:
def get_core_seq(row):
    seq = row['promoter_seq']
    s, e = row['start'], row['stop']
    left = min(s, e) - 1 - extend_range
    right = max(s, e) + extend_range
    left = max(left, 0)
    right = min(right, len(seq))
    return seq[left:right]

fimo_df_split['core_seq'] = fimo_df_split.apply(get_core_seq, axis=1)
fimo_df_split['core_seq_len'] = fimo_df_split['core_seq'].apply(lambda x: len(x))
fimo_df_split['tf_name_new'] = fimo_df_split['tf_name'].apply(lambda x: x.replace('.','_'))
fimo_df_split['msa_path'] = fimo_df_split['tf_name_new'].map(tf_msa_dct)

In [23]:
fimo_df_split['tf_seq'] = fimo_df_split['tf_name'].map(tf_seqs)
fimo_df_split['tf_seq'] = fimo_df_split['tf_seq'].apply(lambda x: ''.join(x))
fimo_df_split

,sequence_name,start,stop,strand,score,pvalue,qvalue,motif_id,promoter_seq,tf_name,boltz_name,core_seq,core_seq_len,tf_name_new,msa_path,tf_seq
0,RbS1,90,77,-,15.000000,0.000005,0.059698,M07129_3.00,AAATATTAATCTAATTTGCCTTTTAGAATTAAGCTTCTTTGGACAG...,Rb01G000105.mRNA1,Rb01G000105.mRNA1-M07129_3.00-RbS1-90-77,TTAGTGCAGCAACTTGTCCAGGAA,24,Rb01G000105_mRNA1,data/MSA_DATABASE/xqw_msa/Rb01G000105_mRNA1/0.a3m,MSSLLYSPNFKSSEAELRKNHAEFLDSSNGFQQLQSQQQQQQSSGL...
1,RbS1,90,77,-,15.000000,0.000005,0.059698,M07129_3.00,AAATATTAATCTAATTTGCCTTTTAGAATTAAGCTTCTTTGGACAG...,Rb03G003089.mRNA1,Rb03G003089.mRNA1-M07129_3.00-RbS1-90-77,TTAGTGCAGCAACTTGTCCAGGAA,24,Rb03G003089_mRNA1,data/MSA_DATABASE/xqw_msa/Rb03G003089_mRNA1/0.a3m,MYNSAPPIKSWQTELPLMDSNTLLNLEHQQPHHQPNSGLLRFRSAP...
2,RbS1,90,77,-,15.000000,0.000005,0.059698,M07129_3.00,AAATATTAATCTAATTTGCCTTTTAGAATTAAGCTTCTTTGGACAG...,Rb04G002309.mRNA1,Rb04G002309.mRNA1-M07129_3.00-RbS1-90-77,TTAGTGCAGCAACTTGTCCAGGAA,24,Rb04G002309_mRNA1,data/MSA_DATABASE/xqw_msa/Rb04G002309_mRNA1/0.a3m,MYPSSSSSSSQKPMGPSPTGITRYVSAPGSLLTSAVDSVIGAVAER...
3,RbS1,90,77,-,15.000000,0.000005,0.059698,M07129_3.00,AAATATTAATCTAATTTGCCTTTTAGAATTAAGCTTCTTTGGACAG...,Rb05G007178.mRNA1,Rb05G007178.mRNA1-M07129_3.00-RbS1-90-77,TTAGTGCAGCAACTTGTCCAGGAA,24,Rb05G007178_mRNA1,data/MSA_DATABASE/xqw_msa/Rb05G007178_mRNA1/0.a3m,MESDLQQPHHKTQQHMNSSLLRYRSAPSSYFANVLDTEFNEPFFNR...
4,RbS1,122,132,+,13.432432,0.000007,0.064092,M07581_3.00,AAATATTAATCTAATTTGCCTTTTAGAATTAAGCTTCTTTGGACAG...,Rb05G000664.mRNA1,Rb05G000664.mRNA1-M07581_3.00-RbS1-122-132,ATTTGAGGTTACGTATGTATG,21,Rb05G000664_mRNA1,data/MSA_DATABASE/xqw_msa/Rb05G000664_mRNA1/0.a3m,MLSIRQVLVDVDKNVIYASPLLLLPNSSSPIYAYIYFFFIFITFDS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,RbS2,1742,1733,-,15.162500,0.000007,0.043372,M02431_3.00,ACCCAAAACCCCATTATCCCAAATTCAACAAATTTTGGTAAAACAA...,Rb04G003734.mRNA1,Rb04G003734.mRNA1-M02431_3.00-RbS2-1742-1733,TGTAGAGGTAGGTATATAAT,20,Rb04G003734_mRNA1,data/MSA_DATABASE/xqw_msa/Rb04G003734_mRNA1/0.a3m,MGRAPCCEKVGLKRGRWTAEEDEILTNYIQLHGEGSWRSLPKNAGL...
267,RbS2,1742,1733,-,15.162500,0.000007,0.043372,M02431_3.00,ACCCAAAACCCCATTATCCCAAATTCAACAAATTTTGGTAAAACAA...,Rb07G006166.mRNA1,Rb07G006166.mRNA1-M02431_3.00-RbS2-1742-1733,TGTAGAGGTAGGTATATAAT,20,Rb07G006166_mRNA1,data/MSA_DATABASE/xqw_msa/Rb07G006166_mRNA1/0.a3m,MGRAPCCEKVGLKKGRWTAEEDQTLINYIHANGEGSWRSLPKNAGL...
268,RbS2,1828,1837,+,12.616279,0.000007,0.079306,M10555_3.00,ACCCAAAACCCCATTATCCCAAATTCAACAAATTTTGGTAAAACAA...,Rb05G007628.mRNA1,Rb05G007628.mRNA1-M10555_3.00-RbS2-1828-1837,TTTTGCTATAAAAAGGCTTT,20,Rb05G007628_mRNA1,data/MSA_DATABASE/xqw_msa/Rb05G007628_mRNA1/0.a3m,MAEPGLEGSGSQPVDLKQHPSGIVPTLQNIVSTVNLDCKLDLKAIA...
269,RbS2,1828,1837,+,12.825581,0.000009,0.099324,M10556_3.00,ACCCAAAACCCCATTATCCCAAATTCAACAAATTTTGGTAAAACAA...,Rb05G007628.mRNA1,Rb05G007628.mRNA1-M10556_3.00-RbS2-1828-1837,TTTTGCTATAAAAAGGCTTT,20,Rb05G007628_mRNA1,data/MSA_DATABASE/xqw_msa/Rb05G007628_mRNA1/0.a3m,MAEPGLEGSGSQPVDLKQHPSGIVPTLQNIVSTVNLDCKLDLKAIA...


In [33]:
# from utils.io_utils import to_boltz_fasta
to_boltz_fasta(fimo_df_split,'input_other/dma/boltz_scan_input')

Generating Boltz FASTAs: 100%|██████████| 271/271 [00:19<00:00, 13.64it/s]


Complete: 271 Boltz FASTA files generated at input_other/dma/boltz_scan_input/BOLTZ_INPUT


### 处理boltz predict后的结果，计算ipsae指标

In [51]:
boltzscan_dir = Path('res/test_dma1/boltz_results_BOLTZ_INPUT/predictions')
res_dct = {}
for d in boltzscan_dir.iterdir():
    for f in d.glob("*.json"):
        with open(f, 'r') as f:
            data = json.load(f)
        res_dct[d.parts[-1]] = data
            
BOLTZSCAN_RES = pd.DataFrame(res_dct).T

In [52]:
need_score = BOLTZSCAN_RES[['confidence_score','ptm', 'iptm']]

In [53]:
BOLTZSCAN_RES

,confidence_score,ptm,iptm,ligand_iptm,protein_iptm,complex_plddt,complex_iplddt,complex_pde,complex_ipde,chains_ptm,pair_chains_iptm
Rb07G005478.mRNA1-M07389_3.00-RbS2-366-352,0.711156,0.673401,0.919932,0.0,0.0,0.658961,0.803478,0.555806,0.425025,"{'0': 0.7368459701538086, '1': 0.9711585640907...","{'0': {'0': 0.7368459701538086, '1': 0.5690283..."
Rb04G005323.mRNA1-M12729_3.00-RbS10-383-391,0.530697,0.459804,0.83933,0.0,0.0,0.453539,0.610021,3.294337,1.424315,"{'0': 0.4331602454185486, '1': 0.9112369418144...","{'0': {'0': 0.4331602454185486, '1': 0.4048565..."
Rb03G003797.mRNA1-M02432_3.00-RbS10-1491-1482,0.484864,0.419894,0.85623,0.0,0.0,0.392022,0.598674,4.275499,2.124665,"{'0': 0.3836081027984619, '1': 0.9308241009712...","{'0': {'0': 0.3836081027984619, '1': 0.3659750..."
Rb01G004746.mRNA1-M07035_3.00-RbS1-1469-1459,0.42948,0.531696,0.563185,0.0,0.0,0.396053,0.618708,2.824561,1.868786,"{'0': 0.5297458171844482, '1': 0.9050256013870...","{'0': {'0': 0.5297458171844482, '1': 0.4107258..."
Rb01G000327.mRNA1-M07397_3.00-RbS2-349-367,0.625181,0.664003,0.80461,0.0,0.0,0.580323,0.76017,1.387919,0.866495,"{'0': 0.6585524082183838, '1': 0.9056137204170...","{'0': {'0': 0.6585524082183838, '1': 0.6026132..."
...,...,...,...,...,...,...,...,...,...,...,...
Rb02G007593.mRNA1-M01332_3.00-RbS2-1622-1631,0.644335,0.884559,0.974099,0.0,0.0,0.561894,0.720682,0.517992,0.345292,"{'0': 0.8729766011238098, '1': 0.9757105708122...","{'0': {'0': 0.8729766011238098, '1': 0.8730056..."
Rb07G000074.mRNA1-xqw_Rb07G000074.mRNA1_0.0_pwm-RbS10-1490-1482,0.463247,0.355433,0.743581,0.0,0.0,0.393163,0.544338,4.579733,2.780926,"{'0': 0.32300058007240295, '1': 0.902082920074...","{'0': {'0': 0.32300058007240295, '1': 0.301349..."
Rb05G007628.mRNA1-M10555_3.00-RbS2-1828-1837,0.783312,0.893365,0.940924,0.0,0.0,0.743908,0.878115,0.535872,0.689176,"{'0': 0.8878298997879028, '1': 0.9633342027664...","{'0': {'0': 0.8878298997879028, '1': 0.8389211..."
Rb03G000659.mRNA1-xqw_Rb03G000659.mRNA1_0.15_pwm-RbS10-806-813,0.656465,0.825215,0.872293,0.0,0.0,0.602508,0.759885,0.840129,0.580674,"{'0': 0.8400059938430786, '1': 0.9593712687492...","{'0': {'0': 0.8400059938430786, '1': 0.7247868..."


In [43]:
def batch_calc_ipsae(res_dir):
    for d in tqdm(res_dir.iterdir(), desc='calc_ipsae'):
        fs = list(d.iterdir())
        for f in fs:
            if f.suffix == '.cif':
                cif = f
            elif 'pae' in f.name:
                pae = f
        calc_ipsae(cif_file=cif, pae_file=pae)

batch_calc_ipsae(boltzscan_dir)

calc_ipsae: 1it [00:00,  4.62it/s]

DEBUG: plddt_boltz1 shape: (287,), token_array shape: (287,)
DEBUG: plddt_boltz1 shape: (520,), token_array shape: (520,)


calc_ipsae: 2it [00:00,  2.36it/s]

DEBUG: plddt_boltz1 shape: (521,), token_array shape: (521,)


calc_ipsae: 3it [00:01,  2.11it/s]

...

calc_ipsae: 53it [00:22,  2.41it/s]

DEBUG: plddt_boltz1 shape: (1059,), token_array shape: (1059,)


calc_ipsae: 55it [00:24,  1.50it/s]

DEBUG: plddt_boltz1 shape: (305,), token_array shape: (305,)
DEBUG: plddt_boltz1 shape: (382,), token_array shape: (382,)


calc_ipsae: 269it [01:58,  2.28it/s]


In [54]:
from utils.io_utils import get_boltz_res
from utils.utils import read_ipsae

ipsae_d = get_boltz_res(boltzscan_dir, res_type='ipsae')
ipsae_res = {}
for k,v in ipsae_d.items():
    ipsae_res[k] = read_ipsae(v)


/home/zlab/boltzscan/utils/utils.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/home/zlab/boltzscan/utils/utils.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/home/zlab/boltzscan/utils/utils.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/home/zlab/boltzscan/utils/utils.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/home/zlab/boltzscan/utils/utils.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/home/zlab/boltzscan/utils/utils.py:42: Futur

In [55]:
ipsae_value_dct = {}
pdcok_value_dct = {}
for i in ipsae_res.keys():
    d = ipsae_res[i]
    
    ipsae_value_dct[i] = d[(d['Type'] == 'max') & (d['Chn1']=='A')]['ipSAE'].min().item()
    pdcok_value_dct[i] = d[(d['Type'] == 'max') & (d['Chn1']=='A')]['pDockQ'].max().item()

In [56]:
need_score['ipsae'] = need_score.index.map(ipsae_value_dct)
need_score['pDockQ'] = need_score.index.map(pdcok_value_dct)

need_score

/tmp/ipykernel_3686312/905192087.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need_score['ipsae'] = need_score.index.map(ipsae_value_dct)
/tmp/ipykernel_3686312/905192087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need_score['pDockQ'] = need_score.index.map(pdcok_value_dct)


,confidence_score,ptm,iptm,ipsae,pDockQ
Rb07G005478.mRNA1-M07389_3.00-RbS2-366-352,0.711156,0.673401,0.919932,0.590752,0.0557
Rb04G005323.mRNA1-M12729_3.00-RbS10-383-391,0.530697,0.459804,0.83933,0.345599,0.0781
Rb03G003797.mRNA1-M02432_3.00-RbS10-1491-1482,0.484864,0.419894,0.85623,0.363522,0.0843
Rb01G004746.mRNA1-M07035_3.00-RbS1-1469-1459,0.42948,0.531696,0.563185,0.048542,0.0269
Rb01G000327.mRNA1-M07397_3.00-RbS2-349-367,0.625181,0.664003,0.80461,0.415862,0.0654
...,...,...,...,...,...
Rb02G007593.mRNA1-M01332_3.00-RbS2-1622-1631,0.644335,0.884559,0.974099,0.869834,0.0747
Rb07G000074.mRNA1-xqw_Rb07G000074.mRNA1_0.0_pwm-RbS10-1490-1482,0.463247,0.355433,0.743581,0.151430,0.0555
Rb05G007628.mRNA1-M10555_3.00-RbS2-1828-1837,0.783312,0.893365,0.940924,0.718933,0.3063
Rb03G000659.mRNA1-xqw_Rb03G000659.mRNA1_0.15_pwm-RbS10-806-813,0.656465,0.825215,0.872293,0.555213,0.0982


In [70]:
need_score['TF'] = need_score.index.str.split('-').str[0]
need_score['TG'] = need_score.index.str.split('-').str[-3]
for tg, df in need_score.groupby('TG'):
    df = df.loc[:,['ipsae','TF','TG']].sort_values(by='ipsae' ,ascending=False)
    df.to_csv(f'input_other/dma/boltzscan_score_{tg}.csv')

/tmp/ipykernel_3686312/1251647675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need_score['TF'] = need_score.index.str.split('-').str[0]
/tmp/ipykernel_3686312/1251647675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  need_score['TG'] = need_score.index.str.split('-').str[-3]
